# Entrenamiento de un Modelo de Deep Learning de Inicio a Fin

Este notebook cubre el proceso completo de entrenamiento de un modelo de deep learning, incluyendo:
- Carga y visualización del dataset
- Preparación de dataloaders
- Definición del modelo
- Entrenamiento
- Validación y evaluación
- Fine-tuning
- Guardado y reutilización del modelo

## Instalación de las librerías necesarias

Ejecuta esta celda si estás en Google Colab o necesitas instalar las dependencias:

In [ ]:
# !pip install torch torchvision numpy matplotlib scikit-learn seaborn pandas tqdm

## 1. Cargar y visualizar el dataset

Utilizaremos el dataset MNIST para este taller, que contiene imágenes de dígitos escritos a mano.

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm

# Verificar si GPU está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

In [ ]:
# Definir transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Cargar datos de entrenamiento y prueba
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

print("Tamaño del set de entrenamiento:", len(train_data))
print("Tamaño del set de prueba:", len(test_data))

In [ ]:
# Visualizar algunos ejemplos
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    image, label = train_data[i]
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Etiqueta: {label}")
    plt.axis('off')
plt.tight_layout()
plt.savefig('../resultados/ejemplos_mnist.png')
plt.show()

## 2. Preparar los dataloaders

Dividimos los datos de entrenamiento en conjuntos de entrenamiento y validación. Luego creamos los dataloaders para cada conjunto.

In [ ]:
from torch.utils.data import random_split, DataLoader

# Dividir el conjunto de entrenamiento en entrenamiento y validación (80% - 20%)
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_subset, val_subset = random_split(train_data, [train_size, val_size])

# Definir el tamaño del batch
batch_size = 64

# Crear los dataloaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

print(f"Tamaño del conjunto de entrenamiento: {train_size} muestras")
print(f"Tamaño del conjunto de validación: {val_size} muestras")
print(f"Tamaño del conjunto de prueba: {len(test_data)} muestras")
print(f"Número de batches de entrenamiento: {len(train_loader)}")
print(f"Número de batches de validación: {len(val_loader)}")
print(f"Número de batches de prueba: {len(test_loader)}")

## 3. Definir el modelo

Creamos una red neuronal simple para la clasificación de dígitos.

In [ ]:
import torch.nn as nn

# Definir el modelo
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Crear una instancia del modelo
model = SimpleNN().to(device)
print(model)

## 4. Configurar función de pérdida y optimizador

In [ ]:
import torch.optim as optim

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## 5. Entrenar el modelo

Implementamos el bucle de entrenamiento y validación.

In [ ]:
# Función para entrenar el modelo
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    for epoch in range(epochs):
        # Fase de entrenamiento
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Barra de progreso para el entrenamiento
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        
        for images, labels in train_pbar:
            images, labels = images.to(device), labels.to(device)
            
            # Poner a cero los gradientes
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass y optimización
            loss.backward()
            optimizer.step()
            
            # Calcular estadísticas
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Actualizar la barra de progreso
            train_pbar.set_postfix({'loss': loss.item(), 'acc': 100 * correct / total})
        
        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        
        # Fase de validación
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        
        # No calcular gradientes en la validación
        with torch.no_grad():
            # Barra de progreso para la validación
            val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Val]')
            
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Calcular estadísticas
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Actualizar la barra de progreso
                val_pbar.set_postfix({'loss': loss.item(), 'acc': 100 * correct / total})
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * correct / total
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    return train_losses, val_losses, train_accs, val_accs

In [ ]:
# Entrenar el modelo
epochs = 10
train_losses, val_losses, train_accs, val_accs = train_model(
    model, train_loader, val_loader, criterion, optimizer, epochs, device
)

In [ ]:
# Visualizar las curvas de pérdida y precisión
plt.figure(figsize=(12, 5))

# Gráfica de pérdida
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_losses, label='Entrenamiento')
plt.plot(range(1, epochs+1), val_losses, label='Validación')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.title('Curva de Pérdida')
plt.legend()
plt.grid(True)

# Gráfica de precisión
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), train_accs, label='Entrenamiento')
plt.plot(range(1, epochs+1), val_accs, label='Validación')
plt.xlabel('Epochs')
plt.ylabel('Precisión (%)')
plt.title('Curva de Precisión')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('../resultados/curva_loss.png')
plt.show()

## 6. Validación y evaluación del modelo

### 6.1 Evaluación en el conjunto de prueba

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc='Evaluando'):
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # Calcular el informe de clasificación
    report = classification_report(all_labels, all_preds, digits=4)
    
    # Calcular la matriz de confusión
    cm = confusion_matrix(all_labels, all_preds)
    
    return report, cm, all_preds, all_labels

In [ ]:
# Evaluar el modelo en el conjunto de prueba
report, cm, test_preds, test_labels = evaluate_model(model, test_loader, device)
print("Informe de clasificación:\n", report)

In [ ]:
# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.tight_layout()
plt.savefig('../resultados/confusion_matrix.png')
plt.show()

### 6.2 K-Fold Cross Validation

Implementaremos una validación cruzada de 5 pliegues (5-fold) para evaluar la robustez del modelo.

In [ ]:
from sklearn.model_selection import KFold

def kfold_cross_validation(dataset, k=5, batch_size=64, epochs=3, device='cpu'):
    # Preparar los datos para k-fold
    X = torch.stack([dataset[i][0] for i in range(len(dataset))])
    y = torch.tensor([dataset[i][1] for i in range(len(dataset))])
    
    # Inicializar K-Fold
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    # Almacenar los resultados de cada fold
    fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold+1}/{k}")
        
        # Crear subconjuntos de datos para este fold
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Crear DataLoaders
        train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
        val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        
        # Crear un nuevo modelo para este fold
        fold_model = SimpleNN().to(device)
        fold_optimizer = optim.Adam(fold_model.parameters(), lr=0.001)
        fold_criterion = nn.CrossEntropyLoss()
        
        # Entrenar el modelo para este fold
        _, _, _, _ = train_model(
            fold_model, train_loader, val_loader, fold_criterion, fold_optimizer, epochs, device
        )
        
        # Evaluar el modelo en el conjunto de validación
        fold_model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = fold_model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f"Fold {fold+1} Accuracy: {accuracy:.2f}%")
        fold_results.append(accuracy)
    
    # Calcular estadísticas generales
    mean_accuracy = sum(fold_results) / len(fold_results)
    std_accuracy = np.std(fold_results)
    
    print(f"\nK-Fold Cross Validation Results:")
    print(f"Mean Accuracy: {mean_accuracy:.2f}%")
    print(f"Standard Deviation: {std_accuracy:.2f}%")
    
    return fold_results, mean_accuracy, std_accuracy

In [ ]:
# Ejecutar K-Fold Cross Validation (con menos epochs para reducir tiempo)
fold_results, mean_accuracy, std_accuracy = kfold_cross_validation(
    train_data, k=5, batch_size=64, epochs=3, device=device
)

In [ ]:
# Visualizar los resultados de la validación cruzada
plt.figure(figsize=(10, 6))
plt.bar(range(1, 6), fold_results, color='skyblue')
plt.axhline(mean_accuracy, color='red', linestyle='--', label=f'Media: {mean_accuracy:.2f}%')
plt.xlabel('Fold')
plt.ylabel('Precisión (%)')
plt.title('Resultados de la Validación Cruzada K-Fold')
plt.xticks(range(1, 6))
plt.ylim(0, 100)
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('../resultados/kfold_results.png')
plt.show()

## 7. Fine-Tuning con modelo preentrenado

Utilizaremos ResNet18 preentrenado y lo adaptaremos para nuestro problema de clasificación de MNIST.

In [ ]:
from torchvision import models

# Cargar modelo preentrenado
model_ft = models.resnet18(pretrained=True)

# Congelar todas las capas
for param in model_ft.parameters():
    param.requires_grad = False

# Modificar la primera capa para aceptar imágenes en escala de grises (1 canal)
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Reemplazar la capa final para tener 10 clases de salida (dígitos 0-9)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

# Mover el modelo a GPU si está disponible
model_ft = model_ft.to(device)
print(model_ft)

In [ ]:
# Entrenar solo la capa final (feature extraction)
optimizer_ft = optim.Adam(model_ft.fc.parameters(), lr=1e-4)
criterion_ft = nn.CrossEntropyLoss()

# Entrenar con feature extraction
epochs_ft = 5
train_losses_ft, val_losses_ft, train_accs_ft, val_accs_ft = train_model(
    model_ft, train_loader, val_loader, criterion_ft, optimizer_ft, epochs_ft, device
)

In [ ]:
# Evaluar el modelo feature extraction en el conjunto de prueba
report_ft, cm_ft, test_preds_ft, test_labels_ft = evaluate_model(model_ft, test_loader, device)
print("Informe de clasificación (Feature Extraction):\n", report_ft)

In [ ]:
# Ahora, descongelar todas las capas para hacer fine-tuning completo
for param in model_ft.parameters():
    param.requires_grad = True

# Usar una tasa de aprendizaje más pequeña para el fine-tuning
optimizer_full_ft = optim.Adam(model_ft.parameters(), lr=1e-5)

# Entrenar con fine-tuning completo
epochs_full_ft = 5
train_losses_full_ft, val_losses_full_ft, train_accs_full_ft, val_accs_full_ft = train_model(
    model_ft, train_loader, val_loader, criterion_ft, optimizer_full_ft, epochs_full_ft, device
)

In [ ]:
# Evaluar el modelo con fine-tuning completo en el conjunto de prueba
report_full_ft, cm_full_ft, test_preds_full_ft, test_labels_full_ft = evaluate_model(model_ft, test_loader, device)
print("Informe de clasificación (Fine-Tuning Completo):\n", report_full_ft)

In [ ]:
# Comparar los resultados: modelo simple vs. feature extraction vs. fine-tuning completo
plt.figure(figsize=(15, 5))

# Gráfica de precisión de entrenamiento
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_accs, label='Modelo Simple')
plt.plot(range(1, epochs_ft+1), train_accs_ft, label='Feature Extraction')
plt.plot(range(1, epochs_full_ft+1), train_accs_full_ft, label='Fine-Tuning Completo')
plt.xlabel('Epochs')
plt.ylabel('Precisión de Entrenamiento (%)')
plt.title('Comparación de Precisión de Entrenamiento')
plt.legend()
plt.grid(True)

# Gráfica de precisión de validación
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), val_accs, label='Modelo Simple')
plt.plot(range(1, epochs_ft+1), val_accs_ft, label='Feature Extraction')
plt.plot(range(1, epochs_full_ft+1), val_accs_full_ft, label='Fine-Tuning Completo')
plt.xlabel('Epochs')
plt.ylabel('Precisión de Validación (%)')
plt.title('Comparación de Precisión de Validación')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('../resultados/comparacion_precisiones.png')
plt.show()

In [ ]:
# Crear un DataFrame con las métricas para cada modelo
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, precision, recall, f1

# Calcular métricas para cada modelo
metrics_simple = calculate_metrics(test_labels, test_preds)
metrics_ft = calculate_metrics(test_labels_ft, test_preds_ft)
metrics_full_ft = calculate_metrics(test_labels_full_ft, test_preds_full_ft)

# Crear DataFrame
metrics_df = pd.DataFrame({
    'Modelo': ['Simple', 'Feature Extraction', 'Fine-Tuning Completo'],
    'Exactitud': [metrics_simple[0], metrics_ft[0], metrics_full_ft[0]],
    'Precisión': [metrics_simple[1], metrics_ft[1], metrics_full_ft[1]],
    'Recall': [metrics_simple[2], metrics_ft[2], metrics_full_ft[2]],
    'F1-Score': [metrics_simple[3], metrics_ft[3], metrics_full_ft[3]]
})

# Mostrar y guardar métricas
print(metrics_df)
metrics_df.to_csv('../resultados/comparacion_metrics.csv', index=False)

## 8. Guardar y reutilizar el modelo

Guardamos los modelos entrenados para su uso futuro.

In [ ]:
# Guardar el modelo simple
torch.save(model.state_dict(), '../modelos/modelo_simple.pth')

# Guardar el modelo con fine-tuning completo
torch.save(model_ft.state_dict(), '../modelos/modelo_fine_tuned.pth')

print("Modelos guardados con éxito en la carpeta 'modelos/'")

In [ ]:
# Ejemplo de cómo cargar el modelo guardado
modelo_cargado = SimpleNN()
modelo_cargado.load_state_dict(torch.load('../modelos/modelo_simple.pth'))
modelo_cargado.to(device)
modelo_cargado.eval()

# Verificamos que funciona correctamente
test_image, test_label = test_data[0]
test_image = test_image.unsqueeze(0).to(device)  # Añadir dimensión del batch

with torch.no_grad():
    output = modelo_cargado(test_image)
    _, predicted = torch.max(output, 1)

print(f"Etiqueta real: {test_label}")
print(f"Predicción: {predicted.item()}")

# Visualizar la imagen
plt.imshow(test_image.squeeze().cpu(), cmap='gray')
plt.title(f"Etiqueta: {test_label}, Predicción: {predicted.item()}")
plt.axis('off')
plt.show()

## 9. Conclusiones

En este taller hemos completado un ciclo completo de entrenamiento de un modelo de Deep Learning:

1. **Carga y visualización de datos**: Utilizamos el dataset MNIST, visualizamos ejemplos para entender los datos.
2. **Preparación de dataloaders**: Dividimos los datos en conjuntos de entrenamiento, validación y prueba.
3. **Definición del modelo**: Creamos una red neuronal simple para clasificación.
4. **Entrenamiento**: Implementamos un bucle de entrenamiento completo con monitoreo de métricas.
5. **Validación y evaluación**: Evaluamos el modelo con validación estándar y k-fold cross validation.
6. **Fine-tuning**: Utilizamos un modelo preentrenado (ResNet18) y lo adaptamos a nuestro problema.
7. **Guardado del modelo**: Guardamos los modelos entrenados para uso futuro.

Los modelos basados en arquitecturas preentrenadas (feature extraction y fine-tuning) tienden a superar al modelo simple en términos de precisión y velocidad de convergencia, lo que demuestra el valor de la transferencia de aprendizaje incluso en tareas relativamente simples como la clasificación de MNIST.